In [1]:
import logging
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Suppress Prophet / cmdstan logs
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)


In [ ]:
#imports
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from IPython.display import display
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
print("Synthetic Data Generation")
np.random.seed(42)
w = np.arange(1, 201)
df = pd.DataFrame({"Week": w,
"Manufactured_Demand": 100 + 0.5*w + np.random.randint(-15, 20, len(w)),
"Remanufactured_Demand": 60 + 0.3*w + np.random.randint(-15, 20, len(w))
}).round(2)
display(df.head())
print("\nProphet Preprocessing")
df["ds"] = pd.date_range("2022-01-02", periods=len(df), freq="W").date
df = df[["ds", "Manufactured_Demand", "Remanufactured_Demand"]]
display(df.head().style.format({"Manufactured_Demand":"{:.2f}","Remanufactured_Demand":"{:.2f}"}).hide(axis="index"))
data = {"Manufactured": df[["ds", "Manufactured_Demand"]].rename(columns={"Manufactured_Demand": "y"}),"Remanufactured": df[["ds","Remanufactured_Demand"]].rename(columns={"Remanufactured_Demand": "y"})}
# Model Fitting & Cross-Validation
print("MODEL FITTING & CROSS-VALIDATION- MANUFACTURED & REMANUFACTURED")
models = {k: Prophet(weekly_seasonality=True, yearly_seasonality=False).fit(d) for k, d in data.items()}
cvs = {k: cross_validation(m, "364 days", "28 days", "56 days") for k, m in models.items()}
cv = pd.concat({k: v["y"] for k, v in cvs.items()}, axis=1)
cv.columns = ["Manufactured_Demand", "Remanufactured_Demand"]
display(cv.head().assign(Week=lambda x: x.index + 1)[["Week", "Manufactured_Demand", "Remanufactured_Demand"]].
style.format({"Manufactured_Demand": "{:.2f}","Remanufactured_Demand": "{:.2f}"}).hide(axis="index"))
# Performance Metrics
metrics = pd.DataFrame({"Metric": ["MAE", "RMSE", "MAPE", "Coverage", "Bias"],"Manufactured": np.append(performance_metrics(cvs["Manufactured"])[["mae", "rmse", "mape", "coverage"]].mean().values,(cvs["Manufactured"]["yhat"] - cvs["Manufactured"]["y"]).mean() ),"Remanufactured": np.append( performance_metrics(cvs["Remanufactured"])[["mae", "rmse", "mape", "coverage"]].mean().values,(cvs["Remanufactured"]["yhat"] - cvs["Remanufactured"]["y"]).mean())})
display(metrics.style.format({"Manufactured": "{:.2f}", "Remanufactured": "{:.2f}"}).set_caption("Performance Metrics"))
# Forecast & Final Visualization
future_201 = pd.DataFrame({"ds": [df["ds"].iloc[-1] + pd.Timedelta(weeks=1)]})
forecast_manu = models["Manufactured"].predict(future_201)
forecast_reman = models["Remanufactured"].predict(future_201)
print(f"\nPredicted Manufactured Demand (Week 201): {forecast_manu['yhat'].iloc[0]:.2f}")
print(f"Predicted Remanufactured Demand (Week 201): {forecast_reman['yhat'].iloc[0]:.2f}")


Synthetic Data Generation


,Week,Manufactured_Demand,Remanufactured_Demand
0,1,113.5,56.3
1,2,100.0,49.6
2,3,93.5,76.9
3,4,107.0,54.2
4,5,105.5,80.5



Prophet Preprocessing


ds,Manufactured_Demand,Remanufactured_Demand
2022-01-02,113.50,56.30
2022-01-09,100.00,49.60
2022-01-16,93.50,76.90
2022-01-23,107.00,54.20
2022-01-30,105.50,80.50


MODEL FITTING & CROSS-VALIDATION- MANUFACTURED & REMANUFACTURED


15:57:00 - cmdstanpy - INFO - Chain [1] start processing
15:57:00 - cmdstanpy - INFO - Chain [1] done processing
15:57:00 - cmdstanpy - INFO - Chain [1] start processing
15:57:00 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/35 [00:00<?, ?it/s]

15:57:01 - cmdstanpy - INFO - Chain [1] start processing
15:57:01 - cmdstanpy - INFO - Chain [1] done processing
15:57:01 - cmdstanpy - INFO - Chain [1] start processing
15:57:01 - cmdstanpy - INFO - Chain [1] done processing
15:57:01 - cmdstanpy - INFO - Chain [1] start processing
15:57:01 - cmdstanpy - INFO - Chain [1] done processing
15:57:02 - cmdstanpy - INFO - Chain [1] start processing
15:57:02 - cmdstanpy - INFO - Chain [1] done processing
15:57:02 - cmdstanpy - INFO - Chain [1] start processing
15:57:02 - cmdstanpy - INFO - Chain [1] done processing
15:57:02 - cmdstanpy - INFO - Chain [1] start processing
15:57:02 - cmdstanpy - INFO - Chain [1] done processing
15:57:03 - cmdstanpy - INFO - Chain [1] start processing
15:57:03 - cmdstanpy - INFO - Chain [1] done processing
15:57:03 - cmdstanpy - INFO - Chain [1] start processing
15:57:03 - cmdstanpy - INFO - Chain [1] done processing
15:57:03 - cmdstanpy - INFO - Chain [1] start processing
15:57:04 - cmdstanpy - INFO - Chain [1]